In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm import tqdm

# Bird Flocking Behaviour

In [ ]:
class Bird():
    def __init__(self, x, y, vx, vy, dt, max_speed, separation_radius, cohesion_radius, alignment_radius):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.dt = dt
        self.max_speed = max_speed
        self.separation_radius = separation_radius
        self.cohesion_radius = cohesion_radius
        self.alignment_radius = alignment_radius
        
    # Avoid hitting neighbours by moving away from them
    def separation(self, birds):
        separation = [0, 0]
        for bird in birds:
            # If the neighbour is within the separation radius
            if bird != self and (bird.x - self.x)**2 + (bird.y - self.y)**2 < self.separation_radius**2:
                separation[0] += (self.x - bird.x) / self.dt
                separation[1] += (self.y - bird.y) / self.dt
        return separation
    
    # Align velocity with the average velocity of nearby neighbours
    def alignment(self, birds):
        alignment = [0, 0]
        count = 0
        for bird in birds:
            # If the neighbour is within the alignment radius
            if bird != self and (bird.x - self.x)**2 + (bird.y - self.y)**2 < self.alignment_radius**2:
                alignment[0] += bird.vx
                alignment[1] += bird.vy
                count += 1
                
        # Calculate the average alignment
        if count > 0:
            alignment[0] /= count
            alignment[1] /= count
        return alignment
    
    # Move towards the average position of nearby neighbours
    def cohesion(self, birds):
        cohesion = [0, 0]
        count = 0
        for bird in birds:
            # If the neighbour is within the cohesion radius
            if bird != self and (bird.x - self.x)**2 + (bird.y - self.y)**2 < self.cohesion_radius**2:
                cohesion[0] += bird.x
                cohesion[1] += bird.y
                count += 1
                
        # Calculate the average cohesion
        if count > 0:
            cohesion[0] /= count
            cohesion[1] /= count
            cohesion[0] = (cohesion[0] - self.x) / self.dt
            cohesion[1] = (cohesion[1] - self.y) / self.dt
        return cohesion
        
    # Find the new velocity and position
    def move(self, birds):
        # Get the separation, cohesion, and alignment velocity vectors
        separation = self.separation(birds)
        cohesion = self.cohesion(birds)
        alignment = self.alignment(birds)
        self.vx += separation[0] + cohesion[0] + alignment[0]
        self.vy += separation[1] + cohesion[1] + alignment[1]
        
        # Limit the speed
        speed = np.sqrt(self.vx**2 + self.vy**2)
        if speed > self.max_speed:
            self.vx = self.vx * self.max_speed / speed
            self.vy = self.vy * self.max_speed / speed
        self.x += self.vx * self.dt
        self.y += self.vy * self.dt

In [ ]:
class Flock():
    def __init__(self, bird_count, map_width, map_height, separation_radius, cohesion_radius, alignment_radius, dt, max_speed):
        self.bird_count = bird_count
        self.map_width = map_width
        self.map_height = map_height
        self.separation_radius = separation_radius
        self.cohesion_radius = cohesion_radius
        self.alignment_radius = alignment_radius
        self.dt = dt
        self.birds = [Bird(np.random.uniform(0, map_width), 
                           np.random.uniform(0, map_height), 
                           np.random.uniform(-1, 1), 
                           np.random.uniform(-1, 1), 
                           dt, 
                           max_speed,
                           separation_radius, 
                           cohesion_radius, 
                           alignment_radius) for _ in range(bird_count)]
            
    # Update the position and velocity of all birds
    def move(self):
        for bird in self.birds:
            bird.move(self.birds)
            
            # Use a periodic boundary condition which wraps around the map
            bird.x = bird.x % self.map_width
            bird.y = bird.y % self.map_height
        
        # Get the positions of all birds for plotting
        positions = [(bird.x, bird.y, bird.vx, bird.vy) for bird in self.birds]
        return positions

In [ ]:
# Run the simulation and get results for all iterations
def simulate_flock(bird_count=50, map_width=100, map_height=100, separation_radius=10, cohesion_radius=15, alignment_radius=15, dt=0.001, max_speed=1, iterations=100):
    flock = Flock(bird_count, map_width, map_height, separation_radius, cohesion_radius, alignment_radius, dt, max_speed)
    all_positions = []
    for _ in range(iterations):
        positions = flock.move()
        all_positions.append(positions)
    
    def plot_iteration(i=0):
        plt.figure(figsize=(10, 10))
        x_positions = [pos[0] for pos in all_positions[i]]
        y_positions = [pos[1] for pos in all_positions[i]]
        x_velocities = [pos[2] for pos in all_positions[i]]
        y_velocities = [pos[3] for pos in all_positions[i]]
        
        plt.scatter(x_positions, y_positions, color='blue')
        plt.quiver(x_positions, y_positions, x_velocities, y_velocities, color='red')
        plt.xlim(0, map_width)
        plt.ylim(0, map_height)
        plt.title(f"Step {i}")
        plt.show()

    slider = widgets.IntSlider(min=0, max=len(all_positions)-1, step=1, value=0)
    widgets.interact(plot_iteration, i=slider)

In [ ]:
# simulate_flock(bird_count=50, map_width=100, map_height=100, separation_radius=8, dt=0.1, max_speed=10, iterations=200)

Analyze the velocity distribution, the average distance between birds, and how these change with different parameters (e.g., view radius, number of birds)

# Human Crowds

Decision Rules for Human Agents:
Avoidance: Unlike birds, humans have a higher capability to anticipate the paths of others and adjust their walking direction and speed accordingly. This rule involves steering to avoid collisions with oncoming pedestrians while maintaining a comfortable personal space. This could involve slight adjustments in direction or speed when another pedestrian is directly oncoming or within a critical personal space radius.

Speed Adjustment: Humans tend to adjust their walking speed based on the density of the crowd. In denser situations, individuals might slow down to navigate through tight spaces. This rule involves decreasing speed as the density of the immediate area increases, simulating more careful navigation through crowded areas.

Goal-Oriented Movement: Each individual has a specific destination. Unlike the alignment rule for birds, humans on a crosswalk are more likely to maintain a generally straight trajectory towards their goal (the other side of the crosswalk) but with adjustments for avoidance. The individuals can have a vector representing their desired direction and adjust this vector based on the need for avoidance and speed adjustment.

Lane Formation: In many human crowds, especially in situations where people are moving in opposite directions, lanes of movement naturally form as individuals align their movements with those heading in the same direction. You could incorporate a rule that encourages alignment with the direction and speed of nearby individuals moving towards the same goal.

In [ ]:
class Pedestrian():
    def __init__(self, goal, x, y, vx, vy, dt, max_speed, separation_radius, alignment_radius, collision_distance_x, collision_distance_y, crowd_radius):
        self.goal = goal
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.dt = dt
        self.max_speed = max_speed
        self.separation_radius = separation_radius
        self.alignment_radius = alignment_radius
        self.collision_distance_x = collision_distance_x
        self.collision_distance_y = collision_distance_y
        self.crowd_radius = crowd_radius
        
    # Avoid hitting nearby neighbours
    def separation(self, pedestrians):
        separation = [0, 0]
        for pedestrian in pedestrians:
            # If the neighbour is within the separation radius
            if pedestrian != self and (pedestrian.x - self.x)**2 + (pedestrian.y - self.y)**2 < self.separation_radius**2:
                separation[0] += (self.x - pedestrian.x) / self.dt
                separation[1] += (self.y - pedestrian.y) / self.dt
        return separation
    
    # Align velocity with the average velocity of nearby neighbours with the same goal
    def alignment(self, pedestrians):
        alignment = [0, 0]
        count = 0
        for pedestrian in pedestrians:
            # If the neighbour has the same goal and is within the alignment radius
            if pedestrian != self and pedestrian.goal == self.goal and (pedestrian.x - self.x)**2 + (pedestrian.y - self.y)**2 < self.alignment_radius**2:
                alignment[0] += pedestrian.vx
                alignment[1] += pedestrian.vy
                count += 1
                
        # Calculate the average alignment
        if count > 0:
            alignment[0] /= count
            alignment[1] /= count
        return alignment
    
    # Avoid collisions with oncoming pedestrians
    def collision(self, pedestrians):
        collision = [0, 0]
        for pedestrian in pedestrians:
            # If the pedestrian has the opposite goal, is ahead, and within the collision rectangle
            # Tendency to move to the right
            if (pedestrian != self and 
                pedestrian.goal != self.goal and 
                (pedestrian.x - self.x) * self.goal > 0 and 
                abs(pedestrian.x - self.x) < self.collision_distance_x and 
                abs(pedestrian.y - self.y) < self.collision_distance_y):
                
                collision[1] -= self.goal * self.max_speed / self.dt
        return collision
    
    # Calculate max speed based on the number of neighbours
    def curr_max_speed(self, pedestrians):
        count = 0
        for pedestrian in pedestrians:
            # If the neighbour is within the crowd radius
            if pedestrian != self and (pedestrian.x - self.x)**2 + (pedestrian.y - self.y)**2 < self.crowd_radius**2:
                count += 1
        
        # Speed = max speed if no neighbours, 0.5 * max speed if 10 neighbours
        return self.max_speed * max(0.5, 1 - count/10)

In [ ]:
class Crossing():
    def __init__(self, pedestrian_count, map_width, map_height, separation_radius, alignment_radius, collision_distance_x, collision_distance_y, crowd_radius, dt, max_speed):
        self.pedestrian_count = pedestrian_count
        self.map_width = map_width
        self.map_height = map_height
        self.separation_radius = separation_radius
        self.alignment_radius = alignment_radius
        self.collision_distance_x = collision_distance_x
        self.collision_distance_y = collision_distance_y
        self.crowd_radius = crowd_radius
        self.dt = dt
        self.max_speed = max_speed
        
        self.pedestrians = []
        for i in range(pedestrian_count):
            goal = np.random.choice([-1, 1])
            if goal == 1:
                x = np.random.uniform(0, map_width/2)
            else:    
                x = np.random.uniform(map_width/2, map_width)
            y = np.random.uniform(0, map_height)
            self.pedestrians.append(Pedestrian(goal, x, y, 0, 0, dt, max_speed, separation_radius, alignment_radius, collision_distance_x, collision_distance_y, crowd_radius))
            
    # Update the position and velocity of all pedestrians
    def move(self):
        for pedestrian in self.pedestrians:
            separation = pedestrian.separation(self.pedestrians)
            alignment = pedestrian.alignment(self.pedestrians)
            collision = pedestrian.collision(self.pedestrians)
            goal_x = pedestrian.goal * pedestrian.max_speed / pedestrian.dt
            pedestrian.vx += separation[0] + alignment[0] + collision[0] + goal_x
            pedestrian.vy += separation[1] + alignment[1] + collision[1]
            
            # Limit the speed
            speed = np.sqrt(pedestrian.vx**2 + pedestrian.vy**2)
            curr_max_speed = pedestrian.curr_max_speed(self.pedestrians)
            if speed > curr_max_speed:
                pedestrian.vx = pedestrian.vx * curr_max_speed / speed
                pedestrian.vy = pedestrian.vy * curr_max_speed / speed
            pedestrian.x += pedestrian.vx * pedestrian.dt
            pedestrian.y += pedestrian.vy * pedestrian.dt
            
            # Stop the pedestrians at the boundaries
            pedestrian.x = min(self.map_width, max(0, pedestrian.x))
            pedestrian.y = min(self.map_height, max(0, pedestrian.y))
        
        # Get the positions of all pedestrians for plotting
        positions = [(pedestrian.x, pedestrian.y, pedestrian.vx, pedestrian.vy, pedestrian.goal) for pedestrian in self.pedestrians]
        return positions

In [ ]:
# Run the simulation and get results for all iterations
def simulate_crossing(pedestrian_count=50, map_width=100, map_height=100, separation_radius=2, alignment_radius=15, collision_distance_x=5, collision_distance_y=3, crowd_radius=15, dt=0.01, max_speed=1, iterations=100):
    crossing = Crossing(pedestrian_count, map_width, map_height, separation_radius, alignment_radius, collision_distance_x, collision_distance_y, crowd_radius, dt, max_speed)
    all_positions = []
    for _ in tqdm(range(iterations)):
        positions = crossing.move()
        all_positions.append(positions)
    
    def plot_iteration(i=0):
        plt.figure(figsize=(10, 10))
        x_positions = [pos[0] for pos in all_positions[i]]
        y_positions = [pos[1] for pos in all_positions[i]]
        x_velocities = [pos[2] for pos in all_positions[i]]
        y_velocities = [pos[3] for pos in all_positions[i]]
        colours = ['blue' if pos[4] == 1 else 'orange' for pos in all_positions[i]]
        
        plt.scatter(x_positions, y_positions, color=colours)
        plt.quiver(x_positions, y_positions, x_velocities, y_velocities, color='red')
        plt.xlim(0, map_width)
        plt.ylim(0, map_height)
        plt.title(f"Step {i}")
        plt.show()

    slider = widgets.IntSlider(min=0, max=len(all_positions)-1, step=1, value=0)
    widgets.interact(plot_iteration, i=slider)

In [ ]:
simulate_crossing(pedestrian_count=500, map_width=100, map_height=100, dt=1, iterations=200)

Potential areas of analysis:
- Analyze how different densities of crowds affect the flow efficiency and average time to cross the crosswalk.
- Study the conditions under which lanes form more distinctly and how this affects overall crowd dynamics.
- Investigate how variations in individual preferences for personal space and speed affect crowd patterns.